# **Task 1: Conversation Management with Reliable Periodic Summarization**

**Configuration**

In [1]:
import requests
import json

API_KEY = "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"
MODEL = "llama-3.1-8b-instant"
K = 3  # Summarize after every K messages
URL = "https://api.groq.com/openai/v1/chat/completions"

conversation_history = []


**Helper Functions**

In [6]:
def add_message(role, content):
    """Add a message to the conversation history."""
    conversation_history.append({"role": role, "content": content})

def summarize_conversation(conv, max_turns=None, max_chars=None):
    """
    Summarize conversation history in a fixed plain-text format.
    Optional truncation by last n messages or last n characters.
    """
    if max_turns:
        conv = conv[-max_turns:]
    text = "\n".join([f"{m['role']}: {m['content']}" for m in conv])
    if max_chars:
        text = text[-max_chars:]

    # Fixed prompt enforcing plain text template
    prompt = f"""
Summarize the following conversation in plain text. Follow this template exactly:
Issue:
Key Point:
Action:
Next Step:
New Request:
Final Action:

Conversation:
{text}

Do not use **, ##, bullets, or numbering. Keep it plain text.
"""
    payload = {
        "model": MODEL,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant that summarizes conversations clearly and concisely."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.3
    }

    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    response = requests.post(URL, headers=headers, data=json.dumps(payload))
    result = response.json()

    try:
        summary = result['choices'][0]['message']['content']
    except KeyError:
        summary = "Error generating summary"

    return summary


def manage_conversation(role, message):
    """
    Add message to history.
    Perform periodic summarization after every K messages.
    """
    add_message(role, message)
    if len(conversation_history) % K == 0:
        summary = summarize_conversation(conversation_history)
        conversation_history.append({"role": "system", "content": f"Conversation summary: {summary}"})


**Demo Conversation**

In [7]:
samples = [
    ("user", "Hi, I need help with my account."),
    ("assistant", "Sure, what seems to be the problem?"),
    ("user", "I forgot my password."),
    ("assistant", "No worries, I can help you reset it."),
    ("user", "Thanks, how do I do that?"),
    ("assistant", "Please follow the link to reset your password."),
    ("user", "Can you help me check my balance?"),
    ("assistant", "Sure, I can provide that."),
    ("user", "Thank you!")
]

for role, msg in samples:
    manage_conversation(role, msg)


**Summarization Demonstrations**

In [8]:
print("\n=== Summarization with last 2 turns ===")
print(summarize_conversation(conversation_history, max_turns=2))

print("\n=== Summarization with last 100 characters ===")
print(summarize_conversation(conversation_history, max_chars=100))



=== Summarization with last 2 turns ===
Issue: Forgotten password and checking account balance
Key Point: User needs assistance with password recovery and account balance inquiry.
Action: The assistant guided the user through the password recovery process and provided account balance information.
Next Step: None, as the user's issue has been resolved.
New Request: None
Final Action: The user has regained access to their account and received their account balance information.

=== Summarization with last 100 characters ===
Issue: 
User regained access to their account and received account balance information.

Key Point: 
Account access and balance information were successfully restored.

Action: 
User expressed gratitude for the assistance.

Next Step: 
None mentioned.

New Request: 
None mentioned.

Final Action: 
User acknowledged the resolution with a thank you.


# **Task 2: JSON Schema Classification & Information Extraction**


**JSON Schema**

In [12]:
# Keys: name, email, phone, location, age
json_schema = {
    "name": str,
    "email": str,
    "phone": str,
    "location": str,
    "age": int
}

**Sample Chats**

In [13]:
sample_chats = [
    "Hello, my name is Shruti Tiwari. You can contact me at shruti.t@example.com or call me on 9876543210. I live in Mumbai and I am 25 years old.",
    "Hi, I'm Rajesh Kumar. My email is rajesh.kumar@example.com. Phone: 9123456780. I am from Delhi and 30 years old.",
    "Good morning, my name is Anjali Singh. Email: anjali.singh@example.com. Mobile: 9988776655. Location: Bangalore. Age: 28."
]

**Function to Extract Data Using Groq API**

In [17]:
def extract_info_from_chat(chat_text):
    """
    Extract structured info (name, email, phone, location, age) from chat using Groq API.
    Cleans output to remove markdown/code blocks and extra text.
    """
    prompt = f"""
Extract the following information from the chat text:
- name
- email
- phone
- location
- age

Return ONLY valid JSON with these keys. Chat text:
\"\"\"{chat_text}\"\"\"
"""

    payload = {
        "model": MODEL,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant that extracts structured information from text."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0,
        "max_tokens": 300
    }

    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    response = requests.post(URL, headers=headers, data=json.dumps(payload))
    result = response.json()

    try:
        output_text = result['choices'][0]['message']['content']

        # Remove Markdown code block if present
        if output_text.startswith("```"):
            output_text = "\n".join(output_text.split("\n")[1:])  # remove first ``` line
            output_text = output_text.rsplit("```", 1)[0]          # remove ending ```

        # Extract JSON portion only
        match = re.search(r"\{.*\}", output_text, re.DOTALL)
        if match:
            json_text = match.group(0)
            extracted_json = json.loads(json_text)
        else:
            extracted_json = {"error": "No valid JSON found", "raw_output": output_text}

    except Exception as e:
        extracted_json = {"error": str(e), "raw_output": output_text}

    # Ensure age is integer
    if "age" in extracted_json and isinstance(extracted_json["age"], str):
        extracted_json["age"] = int(re.sub(r"\D", "", extracted_json["age"]))

    return extracted_json

**Validation**

In [15]:
def validate_extracted_data(data, schema):
    """
    Validate extracted data against JSON schema.
    Returns True if all keys exist and types match, else False with message.
    """
    if "error" in data:
        return False, data["error"]

    for key, typ in schema.items():
        if key not in data:
            return False, f"Missing key: {key}"
        if typ == int:
            # Convert numeric strings to int
            if not isinstance(data[key], int):
                try:
                    data[key] = int(re.sub(r"\D", "", str(data[key])))
                except:
                    return False, f"Invalid type for key {key}"
        else:
            if not isinstance(data[key], typ):
                return False, f"Invalid type for key {key}"

    return True, "Valid"

**Run extraction and validation on sample chats**

In [18]:
for i, chat in enumerate(sample_chats):
    print(f"\n=== Sample Chat {i+1} ===")
    print("Chat text:", chat)

    extracted = extract_info_from_chat(chat)
    print("Extracted JSON:", extracted)

    is_valid, msg = validate_extracted_data(extracted, json_schema)
    print("Validation:", msg)


=== Sample Chat 1 ===
Chat text: Hello, my name is Shruti Tiwari. You can contact me at shruti.t@example.com or call me on 9876543210. I live in Mumbai and I am 25 years old.
Extracted JSON: {'error': "name 're' is not defined", 'raw_output': '{\n  "name": "Shruti Tiwari",\n  "email": "shruti.t@example.com",\n  "phone": "9876543210",\n  "location": "Mumbai",\n  "age": "25"\n}\n'}
Validation: name 're' is not defined

=== Sample Chat 2 ===
Chat text: Hi, I'm Rajesh Kumar. My email is rajesh.kumar@example.com. Phone: 9123456780. I am from Delhi and 30 years old.
Extracted JSON: {'error': "name 're' is not defined", 'raw_output': '{\n  "name": "Rajesh Kumar",\n  "email": "rajesh.kumar@example.com",\n  "phone": "9123456780",\n  "location": "Delhi",\n  "age": "30"\n}'}
Validation: name 're' is not defined

=== Sample Chat 3 ===
Chat text: Good morning, my name is Anjali Singh. Email: anjali.singh@example.com. Mobile: 9988776655. Location: Bangalore. Age: 28.
Extracted JSON: {'error': "name